<a href="https://colab.research.google.com/github/Howard-hth/GenHKHackthon/blob/main/NinJiom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
sorted_data = {
    "img": [],
    "category": [],
    "product": [],
    "description": [],
    "ingredient": [] # (ingredient: purpose)
}

In [108]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# prepare category map
categories = []

# URL of the webpage you want to scrape
url = 'https://www.ninjiom.com/products/'

# Send a GET request to the URL
response = requests.get(url)
html_content = response.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

divs = soup.find_all('div', class_='flex flex-row flex-wrap mb-8 justify-center')
for index, div in enumerate(divs):
  type_list = div.find_all('a')
  type_name = type_list[0]['href'].split("/")[-1].split("?")[0]
  if type_name:
    categories.append(type_name)
print(categories)

['pei-pa-koa-series', 'traditional-chinese-medicine-series', 'herbal-candy-series', 'heatiness-clearing-herbal-series', 'tea-pairing-series']


In [109]:
def page_retrival(category, sorted_data):
  # Send a GET request to the URL
  url = f'https://www.ninjiom.com/products/{category}?product=0'

  response = requests.get(url)
  html_content = response.text

  # Create a BeautifulSoup object
  soup = BeautifulSoup(html_content, 'html.parser')

  tabs = soup.find_all('div', attrs={'role': 'tabpanel'})
  tab_info = []

  for tab in tabs:
    img = tab.find("img")["src"]
    sorted_data["img"].append(img)

    div = tab.find('div', class_='product-description px-8 pt-4')
    sorted_data["category"].append(category)
    sorted_data["product"].append(div.h4.text)
    sorted_data["description"].append(div.p.text)

    ingredient = tab.find_all('div', class_='h-8 sm:h-12 flex flex-col justify-center')  # Retrieves <p> inside <div>
    ingredient_list = []

    for index, div in enumerate(ingredient):
      if not index % 2:
        ingredient_list.append(div.text)
      else:
        ingredient_list[-1] += f"({div.text})"

    if ingredient_list:
      sorted_data["ingredient"].append(str(ingredient_list))
    else:
      sorted_data["ingredient"].append('N/A')

  return sorted_data

# URL of the webpage you want to scrape
sorted_data_example = {
    "img": [],
    "category": [],
    "product": [],
    "description": [],
    "ingredient": [] # (ingredient: purpose)
}

sorted_data_example = page_retrival('pei-pa-koa-series', sorted_data_example)
print(sorted_data_example)

{'img': ['https://cdn.sanity.io/images/dddbuyhi/production/5a32a051f91c237bfbc4658364bc849c32c85236-1920x799.png', 'https://cdn.sanity.io/images/dddbuyhi/production/5a32a051f91c237bfbc4658364bc849c32c85236-1920x799.png', 'https://cdn.sanity.io/images/dddbuyhi/production/5a32a051f91c237bfbc4658364bc849c32c85236-1920x799.png', 'https://cdn.sanity.io/images/dddbuyhi/production/abed0de18208314f22d3f8d65ecc108f49c72291-4000x1665.png', 'https://cdn.sanity.io/images/dddbuyhi/production/7bde2d8dc274ae4c00478ec92e068e2a9bb77b9b-5401x2251.jpg', 'https://cdn.sanity.io/images/dddbuyhi/production/c8ba76d0d10ce92d39d7ce91a951a84be60480c4-5401x2251.jpg'], 'category': ['pei-pa-koa-series', 'pei-pa-koa-series', 'pei-pa-koa-series', 'pei-pa-koa-series', 'pei-pa-koa-series', 'pei-pa-koa-series'], 'product': ['京都念慈菴蜜煉川貝枇杷膏', '京都念慈菴蜜煉川貝枇杷膏', '京都念慈菴蜜煉川貝枇杷膏', '京都念慈菴蜜煉川貝枇杷膏（便利裝）', '京都念慈菴川貝枇杷膏 - 無糖配方', '京都念慈菴兒童枇杷蜜'], 'description': ['本膏得自清代名醫葉天士真傳，適用於傷風咳嗽、痰多痰稠、氣喘不適、咽喉乾癢及聲音沙啞。可清心降火、潤肺養顏。煙酒過多、睡眠不足、喉痛聲啞，一瓶見效。', '

In [113]:
for category in categories:
  sorted_data = page_retrival(category, sorted_data)

df = pd.DataFrame(sorted_data).drop_duplicates()
df.to_excel("NinJiom.xlsx")